In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

/opt/miniconda3/envs/ESM2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 检查设备
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
batch_size = 1
lr = 5e-5
max_length = 128
epochs = 5

Using device: mps


In [3]:
# 模型和存储路径
model_name = "facebook/esm2_t33_650M_UR50D"
model_path = "./models"

# 加载模型和 tokenizer
Tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=model_path)
model = AutoModel.from_pretrained(model_name, cache_dir=model_path)


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 读取数据
file_path = "./data/df_combined_filtered_grouped.pkl"
df = pd.read_pickle(file_path)
df_non_null_stability = df[df['stability'].notna()]
data = df_non_null_stability[['sequence', 'stability']]
data_dict = {
    "sequence": data['sequence'].tolist(),
    "stability": data['stability'].tolist(),
}

# 数据集类
class SequenceStabilityDataset(Dataset):
    def __init__(self, sequences, stabilities, tokenizer=Tokenizer, max_length=max_length):
        self.sequences = sequences
        self.stabilities = torch.tensor(stabilities, dtype=torch.float32)
        self.encoded_sequences = tokenizer(
            sequences, return_tensors="pt", padding=True, truncation=True, max_length=max_length
        )

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return {
            'sequence': {key: val[idx] for key, val in self.encoded_sequences.items()},
            'stability': self.stabilities[idx]
        }

# 数据集分割
train_sequences, val_sequences, train_stabilities, val_stabilities = train_test_split(
    data_dict['sequence'],
    data_dict['stability'],
    test_size=0.2,
    random_state=42
)

train_dataset = SequenceStabilityDataset(train_sequences, train_stabilities, tokenizer=Tokenizer)
val_dataset = SequenceStabilityDataset(val_sequences, val_stabilities, tokenizer=Tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# 模型定义
class StabilityPredictor(nn.Module):
    def __init__(self, base_model):
        super(StabilityPredictor, self).__init__()
        self.base_model = base_model
        self.regressor = nn.Linear(base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_hidden_state = outputs.last_hidden_state[:, 0, :]
        stability = self.regressor(cls_hidden_state)
        return stability

In [6]:
# 初始化模型
stability_model = StabilityPredictor(model)
stability_model.to(device)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = AdamW(stability_model.parameters(), lr=lr)

/opt/miniconda3/envs/ESM2/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
# 训练和验证
for epoch in range(epochs):
    # 训练阶段
    stability_model.train()
    train_loss = 0.0
    train_progress = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} (Training)")
    for batch in train_progress:
        input_ids = batch['sequence']['input_ids'].to(device)
        attention_mask = batch['sequence']['attention_mask'].to(device)
        labels = batch['stability'].to(device)

        optimizer.zero_grad()
        outputs = stability_model(input_ids, attention_mask)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        avg_batch_loss = train_loss / (len(train_progress))
        train_progress.set_postfix({"Batch Loss": loss.item(), "Avg Loss": avg_batch_loss})
    avg_train_loss = train_loss / len(train_dataloader)

    # 验证阶段
    stability_model.eval()
    val_loss = 0.0
    val_progress = tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{epochs} (Validation)")
    with torch.no_grad():
        for batch in val_progress:
            input_ids = batch['sequence']['input_ids'].to(device)
            attention_mask = batch['sequence']['attention_mask'].to(device)
            labels = batch['stability'].to(device)

            outputs = stability_model(input_ids, attention_mask)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item()
            avg_val_batch_loss = val_loss / (len(val_progress))
            val_progress.set_postfix({"Batch Loss": loss.item(), "Avg Loss": avg_val_batch_loss})
    avg_val_loss = val_loss / len(val_dataloader)

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# 保存模型
save_path = "./stability_model.pth"
torch.save(stability_model.state_dict(), save_path)
print(f"Model saved to {save_path}")

Epoch 1/5 (Training):   0%|          | 0/79219 [00:00<?, ?it/s]/opt/miniconda3/envs/ESM2/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/5 (Training):   0%|          | 2/79219 [00:04<44:33:19,  2.02s/it, Batch Loss=1.2, Avg Loss=2.1e-5]   


KeyboardInterrupt: 

In [ ]:
# 评估模型性能
stability_model.eval()
predictions = []
true_values = []
with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['sequence']['input_ids'].to(device)
        attention_mask = batch['sequence']['attention_mask'].to(device)
        labels = batch['stability'].to(device)

        outputs = stability_model(input_ids, attention_mask)
        predictions.extend(outputs.squeeze().cpu().numpy())
        true_values.extend(labels.cpu().numpy())

mse = mean_squared_error(true_values, predictions)
r2 = r2_score(true_values, predictions)
print(f"Mean Squared Error: {mse:.4f}, R^2 Score: {r2:.4f}")

mse = mean_squared_error(true_values, predictions)
r2 = r2_score(true_values, predictions)
print(f"Mean Squared Error: {mse:.4f}, R^2 Score: {r2:.4f}")

In [ ]:
# 预测稳定性
sequence = "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"
encoded_sequence = Tokenizer(sequence, return_tensors="pt")
input_ids = encoded_sequence['input_ids'].to(device)
attention_mask = encoded_sequence['attention_mask'].to(device)
stability = stability_model(input_ids, attention_mask).item()
print(f"Predicted Stability: {stability:.4f}")